In [ ]:
!pip install ultralytics -q
!pip install scikit-learn numpy opencv-python tensorflow spacy -q
!pip install gdown==4.6.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.7/901.7 kB 52.7 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/wjnwjn59/deep_sort.git

Cloning into 'deep_sort'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 167 (delta 9), reused 7 (delta 7), pack-reused 150 (from 2)
Receiving objects: 100% (167/167), 77.68 KiB | 12.95 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [ ]:
!gdown --no-check-certificate --folder https://drive.google.com/open?id=18fKzfqnqhqW3s9zwsCbnVJ5XF2JFeqMp

Retrieving folder list
Retrieving folder 1VVqtL0klSUvLnmBKS89il1EKC3IxUBVK detections
Retrieving folder 1qNWOpUtKG8GqEiL-LbBdXyvifUtcbOvc MOT16_POI_test
Processing file 1aEzvFHPK-N6hqLXMqhh3i9JJzn7WFUA3 MOT16-01.npy
Processing file 1h_ktJDBIEXaSBAA-RxKNYnL9e4fp2HPd MOT16-03.npy
Processing file 1ilOElwfYZLwQKH57HoYdXfuYhpPibfqF MOT16-06.npy
Processing file 1TajzH3GbumKmtYvKBvOtGERFGD0tStwG MOT16-07.npy
Processing file 1WB9Mi4RLVPHV4_20sVq7FdoeG5JYQ_J1 MOT16-08.npy
Processing file 1mksH9GWNT7zmcuq6rlRev8pevZz8Rfsm MOT16-12.npy
Processing file 1FVVhn_IpxQ_jkYhc0CUQHSQMm1SMTEBj MOT16-14.npy
Retrieving folder 1DcOcApOkxP3NdeIUXxVF1KNex6T6YDq3 MOT16_POI_train
Processing file 1Va__9NWU2ZCmaxIq4oIabi05NYWEOk1K MOT16-02.npy
Processing file 1EH7orgDPp7kqRY5OA0hEctcEtQnYq0Ea MOT16-04.npy
Processing file 1RCfHJx5ZoUecapbZCsgp0tCEiItvLsd8 MOT16-05.npy
Processing file 1VLOvn-mbpY0Q1rsMONQZhaEQIGEmyLQL MOT16-09.npy
Processing file 1SbMhOgYPvZ84xE8lRtXc7CLXJF86lwf4 MOT16-10.npy
Processing file 1a4w-Ho

In [ ]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO

from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections as gdet

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
class YOLO_Detector:
  def __init__ (self,
                model_path):
    self.model = YOLO(model_path)
  def detect (self, source_img):
    results = self.model.predict(source_img, verbose=False)[0]
    bboxes = results.boxes.xywh.cpu().numpy()
    bboxes[:, :2] = bboxes[:, :2] - (bboxes[:, 2:] / 2)
    confidences = results.boxes.conf.cpu().numpy()
    class_ids = results.boxes.cls.cpu().numpy()
    return bboxes, confidences, class_ids

In [ ]:
class deepSORT:
  def __init__(
      self,
      model_path= 'resources/networks/mars-small128.pb',
      max_cosine_distance=0.7,
      nn_budget=None,
      classes=['objects']
  ):
    self.encoder = gdet.create_box_encoder(model_path, batch_size=1)
    self.metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine",
        max_cosine_distance,
        nn_budget
    )
    self.tracker = Tracker(self.metric)

    key_list = []
    val_list  = []
    for ID, class_name in enumerate(classes):
      key_list.append(ID)
      val_list.append(class_name)
    self.key_list = key_list
    self.val_list = val_list
  def tracking (
      self,
      origin_frame,
      bboxes,
      scores,
      class_ids
  ):
    features = self.encoder(origin_frame, bboxes)
    detections = [
        Detection(bbox, score, class_id, feature) for bbox, score, class_id, feature in zip(bboxes, scores, class_ids, features)]

    self.tracker.predict()
    self.tracker.update(detections)

    tracked_bboxes = []
    for track in self.tracker.tracks:
      if not track.is_confirmed() or track.time_since_update > 3:
        continue
      bbox = track.to_tlbr()
      class_id = track.get_class()
      conf_score = track.get_conf_score()
      tracking_id = track.track_id
      tracked_bboxes.append(
                bbox.tolist() + [class_id, conf_score, tracking_id]
            )
    tracked_bboxes = np.array(tracked_bboxes)

    return tracked_bboxes

In [ ]:
def draw_detection(
    img,
    bboxes,
    scores,
    class_ids,
    ids,
    classes=['objects'],
    mask_alpha=0.3
):
    height, width = img.shape[:2]
    np.random.seed(0)
    rng = np.random.default_rng(3)
    colors = rng.uniform(0, 255, size=(len(classes), 3))

    mask_img = img.copy()
    det_img = img.copy()

    size = min([height, width]) * 0.0006
    text_thickness = int(min([height, width]) * 0.001)

    # Draw bounding boxes and labels of detections
    for bbox, score, class_id, id_ in zip(bboxes, scores, class_ids, ids):
        color = colors[class_id]

        x1, y1, x2, y2 = bbox.astype(int)

        # Draw rectangle
        cv2.rectangle(det_img, (x1, y1), (x2, y2), color, 2)

        # Draw fill rectangle in mask image
        cv2.rectangle(mask_img, (x1, y1), (x2, y2), color, -1)

        label = classes[class_id]
        caption = f'{label} {int(score * 100)}% ID: {id_}'
        (tw, th), _ = cv2.getTextSize(text=caption, fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                                    fontScale=size, thickness=text_thickness)
        th = int(th * 1.2)

        cv2.rectangle(det_img, (x1, y1),
                    (x1 + tw, y1 - th), color, -1)
        cv2.rectangle(mask_img, (x1, y1),
                    (x1 + tw, y1 - th), color, -1)
        cv2.putText(det_img, caption, (x1, y1),
                    cv2.FONT_HERSHEY_SIMPLEX, size, (255, 255, 255), text_thickness, cv2.LINE_AA)

        cv2.putText(mask_img, caption, (x1, y1),
                    cv2.FONT_HERSHEY_SIMPLEX, size, (255, 255, 255), text_thickness, cv2.LINE_AA)

    return cv2.addWeighted(mask_img, mask_alpha, det_img, 1 - mask_alpha, 0)

In [ ]:
def video_tracking(
    video_path,
    detector,
    tracker,
    save_result_name,
    is_save_result=False,
    save_dir='tracking_results'
):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if is_save_result:
        os.makedirs(save_dir, exist_ok=True)
        # Get the video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        # Define the codec and create the video writer
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')

        save_result_name = save_result_name + '.avi'
        save_result_path = os.path.join(save_dir, save_result_name)
        out = cv2.VideoWriter(save_result_path, fourcc, fps, (width, height))


    all_tracking_results = []
    tracked_ids = np.array([], dtype=np.int32)
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        detector_results = detector.detect(frame)
        bboxes, scores, class_ids = detector_results

        tracker_pred = tracker.tracking(
            origin_frame=frame,
            bboxes=bboxes,
            scores=scores,
            class_ids=class_ids
        )
        if tracker_pred.size > 0:
            bboxes = tracker_pred[:, :4]

            class_ids = tracker_pred[:, 4].astype(int)
            conf_scores = tracker_pred[:, 5]
            tracking_ids = tracker_pred[:, 6].astype(int)

            # Get new tracking IDs
            new_ids = np.setdiff1d(tracking_ids, tracked_ids)

            # Store new tracking IDs
            tracked_ids = np.concatenate((tracked_ids, new_ids))

            result_img = draw_detection(
                img=frame,
                bboxes=bboxes,
                scores=conf_scores,
                class_ids=class_ids,
                ids=tracking_ids
            )
        else:
            result_img=frame

        all_tracking_results.append(tracker_pred)

        if is_save_result == 1:
            out.write(result_img)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Release video capture
    cap.release()
    if is_save_result:
        out.release()
    cv2.destroyAllWindows()

    return all_tracking_results

In [ ]:
yolo_model_path = '/content/best.pt'

detector = YOLO_Detector(yolo_model_path)
tracker = deepSORT()

In [ ]:
!gdown 1sCJAZn6Ug17HWn7PmwGH1XAqjgPhr8ZQ

Downloading...
From: https://drive.google.com/uc?id=1sCJAZn6Ug17HWn7PmwGH1XAqjgPhr8ZQ
To: /content/CityRoam.mp4
100% 5.43M/5.43M [00:00<00:00, 19.1MB/s]


In [ ]:
video_path = '/content/CityRoam.mp4'
all_tracking_results = video_tracking(
    video_path,
    detector,
    tracker,
    save_result_name='tracking_results.mp4',
    is_save_result=True
)

In [ ]:
import os
import subprocess
from IPython.display import HTML
from base64 import b64encode

def show_tracked_video(output_video_path):
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_video_path)
    os.makedirs(output_dir, exist_ok=True)

    # Define the path for the compressed video
    compressed_path = os.path.join(output_dir, 'result_compressed.mp4')

    # Compress the video using ffmpeg
    ffmpeg_command = [
        'ffmpeg',
        '-i', output_video_path,
        '-vcodec', 'libx264',
        '-crf', '23',  # Adjust this as needed for quality vs. size
        '-preset', 'fast',  # Adjust this for a balance of speed and quality
        compressed_path
    ]

    # Execute ffmpeg command
    result = subprocess.run(ffmpeg_command, capture_output=True, text=True)

    # Check if ffmpeg command executed successfully
    if result.returncode != 0:
        print("Error compressing video with ffmpeg:", result.stderr)
        return

    # Load and display the video if the compression was successful
    if os.path.exists(compressed_path):
        mp4 = open(compressed_path, 'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        return HTML(f"""
        <video width=600 controls>
              <source src="{data_url}" type="video/mp4">
        </video>
        """)
    else:
        print("Compressed video file does not exist.")



In [ ]:
output_video_path = '/content/tracking_results/tracking_result.avi'
display(show_tracked_video(output_video_path))

Error compressing video with ffmpeg: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --en

None

In [ ]:
video_path_2 = '/content/Test_video.mp4'
all_tracking_results_2 = video_tracking(
    video_path_2,
    detector,
    tracker,
    save_result_name= 'test_video_result',
    is_save_result=True
)

In [ ]:
output_video_path_2 = '/content/tracking_results/test_video_result.avi'
display(show_tracked_video(output_video_path_2))

Error compressing video with ffmpeg: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --en

None

In [ ]:
from google.colab import files
files.download('/content/tracking_results/test_video_result.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
display('/content/tracking_results/result_compressed.mp4')

'/content/tracking_results/result_compressed.mp4'